<a href="https://colab.research.google.com/github/mistryishan25/NLU_Research/blob/master/PSAW_attempt.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import pandas as pd
import requests #Pushshift accesses Reddit via an url so this is needed
import json #JSON manipulation
import csv #To Convert final table into a csv file to save to your machine
import time
import datetime

Get the list of submissions based on the subreddits and also try to obtain the ones that are time restricted because this API seems to work well with relavtively older data

In [ ]:
sub_count = 0 # counter for total submissions we have so far
sub_stats = {} # where we will store the data

In [ ]:
# The function to get the endpoint/URL for the specific data - submissions
def get_PSAW_data(subreddit, before, after,size = 1000,num_com = 75,over_18 = False):
  submission_endpoint = 'https://api.pushshift.io/reddit/search/submission/?subreddit='+str(subreddit)+'&size='+str(size)+'&before='+str(before)+'&after='+str(after)+'&over_18='+str(over_18)+'&num_comments=>'+str(num_com)
  print(f"The submission endpoint/URL : {submission_endpoint}")
  submissions = requests.get(submission_endpoint)
  data = json.loads(submissions.text)
  #the submissions stored in the key "data"
  return data['data']

In [ ]:
# Parameters for submission endpoint: subreddit, size, num_comments, before , after, over_18.
def extract_data(submission):
  subm_data = []
  title = submission['title']

  #flairs are not always present so we wrap in try/except
  try:
      flair = submission['link_flair_text']
  except KeyError:
      flair = "NaN"    
  sub_id = submission['id']
  score = submission['score']
  created = datetime.datetime.fromtimestamp(submission['created_utc']) #1520561700.0
  num_comms = submission['num_comments']
  permalink = submission['permalink']
  try:
    text = submission['selftext']
  except KeyError:
    text = "NaN"

  subm_data.append((sub_id,title,score,created,num_comms,permalink,flair, text))
  sub_stats[sub_id] = subm_data # id : [ <features>, ]


For now lets stick to one then we can loop over all the other subreddits.

In [ ]:
# subreddit, size,score, num_comments(has a default), before , after, over_18.
subreddit = 'unexpected' #turn into list afterwards
#size = 200 if we are getting all then might not need this?
#the UNIX timestamps ref from https://www.unixtimestamp.com/index.php 
after = '1577836800' # From : 1/3/20  
before = '1607040000' # To 1/6/20 



In [ ]:
data = get_PSAW_data(subreddit,before,after)

while len(data)>0:
  for submission in data:
    extract_data(submission)
    sub_count+=1

     # Calls getPushshiftData() with the created date of the last submission
    print(len(data))
    print(str(datetime.datetime.fromtimestamp(data[-1]['created_utc'])))
    #updating "after" variable to last created date of submission
    after = data[-1]['created_utc']
    #now the everything after the last submission and before the before date is extracted
    data = get_PSAW_data(subreddit,before,after)

print(len(data))

The submission endpoint/URL : https://api.pushshift.io/reddit/search/submission/?subreddit=unexpected&size=1000&before=1607040000&after=1577836800&over_18=False&num_comments>=75
100
2020-01-01 19:39:39
The submission endpoint/URL : https://api.pushshift.io/reddit/search/submission/?subreddit=unexpected&size=1000&before=1607040000&after=1577907579&over_18=False&num_comments>=75
100
2020-01-02 10:00:21
The submission endpoint/URL : https://api.pushshift.io/reddit/search/submission/?subreddit=unexpected&size=1000&before=1607040000&after=1577959221&over_18=False&num_comments>=75
100
2020-01-03 00:26:55
The submission endpoint/URL : https://api.pushshift.io/reddit/search/submission/?subreddit=unexpected&size=1000&before=1607040000&after=1578011215&over_18=False&num_comments>=75
100
2020-01-03 16:02:33
The submission endpoint/URL : https://api.pushshift.io/reddit/search/submission/?subreddit=unexpected&size=1000&before=1607040000&after=1578067353&over_18=False&num_comments>=75
100
2020-01-04

JSONDecodeError: ignored

In [ ]:
print(f"submissions added = {len(sub_stats)}")
print(f"First submission : {list(sub_stats.values())[0][0][0]} /n Title :{list(sub_stats.values())[0][0][1]} \n Created on : {list(sub_stats.values())[0][0][3]} ")
print((f"Last submission : {list(sub_stats.values())[-1][0][0]} /n Title :{list(sub_stats.values())[-1][0][1]} \n Created on : {list(sub_stats.values())[-1][0][3]} "))

submissions added = 168
First submission : eib0q1 /n Title :Just wait for it 
 Created on : 2020-01-01 00:01:40 
Last submission : fd5ziq /n Title :Say what? 
 Created on : 2020-03-04 02:20:49 


In [ ]:
def update_subs_file():
    upload_count = 0
    #location = "\\Reddit Data\\" >> If you're running this outside of a notebook you'll need this to direct to a specific location
    print("input filename of submission file, please add .csv")
    filename = input() #This asks the user what to name the file
    file = filename
    with open(file, 'w', newline='', encoding='utf-8') as file: 
        a = csv.writer(file, delimiter=',')
        headers = ['sub_id',"title",'score','created',"num_comms","permalink",'flair', 'text']
        a.writerow(headers)
        for sub in sub_stats:
            a.writerow(sub_stats[sub][0])
            upload_count+=1
            
        print(str(upload_count) + " submissions have been uploaded")


In [ ]:
update_subs_file()

In [ ]:
test = pd.read_csv("subs_unexpected.csv")

In [ ]:
test.columns

In [ ]:
test.head()

In [ ]:
len(test.loc[test.num_comms ==0])
# these many have no comments

Drop flair and not mu

In [ ]:
test['flair'].value_counts()

In [ ]:
len(test)

In [ ]:
import matplotlib.pyplot as plt
test['num_comms'].value_counts().hist() 

In [ ]:
len(test['title'].unique())

In [ ]:
len(test['text'].unique())

In [ ]:
#Create function for extracting comments from submission
def extractComments(subpost):
    post = r.submission(id=subpost)
    #Use post id for title
    title = str(subpost)
    allComments = list()
    topDict = dict()
    topCounter = 0
    try:
        post.comments.replace_more(limit=None, threshold=0)
        for comment in post.comments.list(): #All comments
            allComments.append(comment.body) #store all comments in AllComments
    except:
        print("An error has occured with " + str(subpost) + ". Will attempt again in 30 seconds.")
        time.sleep(30)
        post.comments.replace_more(limit=None, threshold=0)
        for comment in post.comments.list(): #All comments
            allComments.append(comment.body) #store all comments in AllComments